In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from cellitaire.environment.agents.PPOMemory import PPOMemory
from torch.distributions.categorical import Categorical

class ActorNetwork(nn.Module):
    def __init__(self, n_actions, input_dims, alpha,
            fc1_dims=2048, fc2_dims=2048, chkpt_dir='tmp/ppo'):
        super(ActorNetwork, self).__init__()

        self.checkpoint_file = os.path.join(chkpt_dir, 'actor_torch_ppo')
        self.actor = nn.Sequential(
                nn.Linear(*input_dims, fc1_dims),
                nn.LayerNorm(fc1_dims),
                nn.ReLU(),
                nn.Linear(fc1_dims, fc2_dims),
                nn.LayerNorm(fc2_dims),
                nn.ReLU(),
                nn.Linear(fc2_dims, n_actions),
                nn.Softmax(dim=-1)
        )

        self.optimizer = optim.Adam(self.parameters(), lr=alpha)
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self, state):
        dist = self.actor(state)
        dist = Categorical(dist)
        
        return dist

    def save_checkpoint(self):
        torch.save(self.state_dict(), self.checkpoint_file)

    def load_checkpoint(self):
        self.load_state_dict(torch.load(self.checkpoint_file))
        
class CriticNetwork(nn.Module):
    def __init__(self, input_dims, alpha, fc1_dims=4096, fc2_dims=4096,
            chkpt_dir='tmp/ppo'):
        super(CriticNetwork, self).__init__()

        self.checkpoint_file = os.path.join(chkpt_dir, 'critic_torch_ppo')
        self.critic = nn.Sequential(
                nn.Linear(*input_dims, fc1_dims),
                nn.LayerNorm(fc1_dims),
                nn.ReLU(),
                nn.Linear(fc1_dims, fc2_dims),
                nn.LayerNorm(fc2_dims),
                nn.ReLU(),
                nn.Linear(fc2_dims, 1)
        )

        self.optimizer = optim.Adam(self.parameters(), lr=alpha)
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self, state):
        value = self.critic(state)

        return value

    def save_checkpoint(self):
        torch.save(self.state_dict(), self.checkpoint_file)

    def load_checkpoint(self):
        self.load_state_dict(torch.load(self.checkpoint_file))

class Agent:
    def __init__(self, n_actions, input_dims, gamma=0.99, alpha=0.0003, gae_lambda=0.95,
            policy_clip=0.2, batch_size=64, n_epochs=10, checkpoint_dir='tmp/ppo'):
        self.gamma = gamma
        self.policy_clip = policy_clip
        self.n_epochs = n_epochs
        self.gae_lambda = gae_lambda
        self.n_actions = n_actions

        self.actor = ActorNetwork(n_actions, input_dims, alpha, chkpt_dir=checkpoint_dir)
        self.critic = CriticNetwork(input_dims, alpha, chkpt_dir=checkpoint_dir)
        self.memory = PPOMemory(batch_size)
       
    def remember(self, state, action, probs, vals, reward, done):
        self.memory.store_memory(state, action, probs, vals, reward, done)

    def save_models(self):
        print('... saving models ...')
        self.actor.save_checkpoint()
        self.critic.save_checkpoint()

    def load_models(self):
        print('... loading models ...')
        self.actor.load_checkpoint()
        self.critic.load_checkpoint()

    def choose_action(self, observation):
        state = torch.tensor(np.array([observation]), dtype=torch.float).to(self.actor.device)

        dist = self.actor(state)
        value = self.critic(state)
        action = dist.sample()

        probs = torch.squeeze(dist.log_prob(action)).item()
        action = torch.squeeze(action).item()
        value = torch.squeeze(value).item()

        return action, probs, value

    def choose_legal_action_mostly(self, observation, legal_actions):
        with torch.no_grad():
            state = torch.tensor(np.array([observation]), dtype=torch.float).to(self.actor.device)
    
            dist = self.actor(state)
            value = self.critic(state)
    
            mask = torch.zeros(self.n_actions).to(self.actor.device)
            mask[legal_actions] = 1
            masked_dist = dist.probs * mask
            masked_dist = Categorical(masked_dist)
            action = masked_dist.sample()

            probs = torch.squeeze(dist.log_prob(action)).item()
            action = torch.squeeze(action).item()
            value = torch.squeeze(value).item()
    
            return action, probs, value

    def learn(self):
        for _ in range(self.n_epochs):
            state_arr, action_arr, old_prob_arr, vals_arr,\
            reward_arr, dones_arr, batches = \
                    self.memory.generate_batches()

            rewards = torch.tensor(reward_arr, dtype=torch.float32, device=self.actor.device)
            values = torch.tensor(vals_arr, dtype=torch.float32, device=self.actor.device)
            dones = torch.tensor(dones_arr, dtype=torch.float32, device=self.actor.device)
            advantage = compute_advantage(values, dones, rewards, self.gamma, self.gae_lambda)

            for batch in batches:
                states = torch.tensor(state_arr[batch], dtype=torch.float).to(self.actor.device)
                old_probs = torch.tensor(old_prob_arr[batch]).to(self.actor.device)
                actions = torch.tensor(action_arr[batch]).to(self.actor.device)

                dist = self.actor(states)
                critic_value = self.critic(states)

                critic_value = torch.squeeze(critic_value)

                new_probs = dist.log_prob(actions)
                prob_ratio = new_probs.exp() / old_probs.exp()
                #prob_ratio = (new_probs - old_probs).exp()
                weighted_probs = advantage[batch] * prob_ratio
                weighted_clipped_probs = torch.clamp(prob_ratio, 1-self.policy_clip,
                        1+self.policy_clip)*advantage[batch]
                actor_loss = -torch.min(weighted_probs, weighted_clipped_probs).mean()

                returns = advantage[batch] + values[batch]
                critic_loss = (returns-critic_value)**2
                critic_loss = critic_loss.mean()

                total_loss = actor_loss + 0.5*critic_loss
                self.actor.optimizer.zero_grad()
                self.critic.optimizer.zero_grad()
                total_loss.backward()
                nn.utils.clip_grad_norm_(self.actor.parameters(), max_norm=1)
                nn.utils.clip_grad_norm_(self.critic.parameters(), max_norm=1)
                self.actor.optimizer.step()
                self.critic.optimizer.step()

        self.memory.clear_memory()               

def compute_advantage(values, dones, rewards, gamma, gae_lambda):
    advantage = torch.zeros_like(rewards, device=values.device)
    T = len(rewards) - 1
    discount = gamma * gae_lambda
    discount_factors = discount ** torch.arange(T + 1, device=values.device, dtype=values.dtype)
    for t in range(T):
        discount_factors = discount_factors[:-1]
        next_vs = values[t+1:]
        masked = (1 - dones[t:-1]) * next_vs
        discounted_mask = masked * gamma
        discounted_mask_plus_rewards = rewards[t:-1] + discounted_mask
        diff = discounted_mask_plus_rewards - values[t:-1]
        discounted_diff = discount_factors * diff
        advantage[t] = discounted_diff.sum()
    return advantage

In [34]:
#from cellitaire.environment.agents.PPOAgent import Agent
from cellitaire.environment.cellitaire_env import CellitaireEnv
from cellitaire.environment.rewards.reward import *
from cellitaire.environment.rewards.foundation_rewards import *
import numpy as np

board_rows = 7
board_cols = 12
num_reserved = 6
test_reward = CombinedReward([
    #PlacedCardInFoundationReward(weight=6),
    WinReward(weight=100, rows=board_rows, cols=board_cols),
    #ConstantReward(weight=0.5),
    ScalingPlacedCardInFoundationReward(weight=1, rows=board_rows, cols=board_cols),
    #PlayedLegalMoveReward(weight=1, rows=board_rows, cols=board_cols, num_reserved = num_reserved),
    #PeriodicPlacedCardInFoundationReward(weight=4, reward_period=3),
    #CreatedMovesReward(weight=1, num_reserved=num_reserved, foundation_count_dropoff=30)
])
env = CellitaireEnv(test_reward, rows=board_rows, cols=board_cols, num_reserved=num_reserved, max_moves=1200, max_illegal_moves=1200)
#env.render()

#agent = Agent(n_actions=env.action_space.n, batch_size=batch_size, 
#                alpha=alpha, n_epochs=n_epochs, 
#                input_dims=env.observation_space.shape)

N = 10000
batch_size = 2000
n_epochs = 5
alpha = 4e-4
gamma = 0.99
gae_lambda = 0.99
checkpoint_dir = 'tmp/scratch_higher_lambda'
agent = Agent(n_actions=env.action_space.n, 
    input_dims=(board_rows * board_cols * 4 + 6,), batch_size=batch_size, 
                alpha=alpha, n_epochs=n_epochs, gamma=gamma, gae_lambda=gae_lambda, checkpoint_dir=checkpoint_dir)

normalize_reward = True
save_model = True

In [35]:
env.action_space.n = board_rows * board_cols

In [ ]:
import os

try:
    agent.load_models()
except:
    print('New run, godspeed lad')
    os.mkdir(checkpoint_dir)
n_games = 300

best_score = 0
score_history = []
cards_saved_history = []
max_score = 0
max_cards_saved = 0
episodes_without_best = 0

learn_iters = 0
avg_score = 0
n_steps = 0
i = 0
while True:
    env.reset()
    observation = env.get_state()
    done = False
    truncated = False
    score = 0
    while not done and not truncated:
        action, prob, val = agent.choose_legal_action_mostly(observation, env.get_legal_actions_as_int())
        observation_, reward, done, truncated, info = env.step(action)
        n_steps += 1
        if normalize_reward:
            score += reward / env.reward.max_reward
        else:
            score += reward
        agent.remember(observation, action, prob, val, reward, done)
        if n_steps % N == 0:
            agent.learn()
            learn_iters += 1
        observation = observation_
    i += 1
    score_history.append(score)
    cards_saved = env.game.foundation.total_cards()
    cards_saved_history.append(cards_saved)
    avg_score = np.mean(score_history[-100:])
    recent_max = np.max(score_history[-110:])
    avg_cards_saved = np.mean(cards_saved_history[-100:])
    recent_max_cs = np.max(cards_saved_history[-100:])

    max_score = max(max_score, score)
    max_cards_saved = max(max_cards_saved, cards_saved)
    
    if avg_score > best_score and n_steps > N:
        best_score = avg_score
        recent_std = np.std(score_history[-100:])
        if save_model:
            agent.save_models()
        episodes_without_best = 0
        print(f'episode {i:>5} | score {score:>6.1f} | avg s {avg_score:>6.1f} | max s R {recent_max:>5.1f} | max s A {max_score:>5.1f} | avg cs {avg_cards_saved:>4.1f} | max cs R {recent_max_cs:>2.0f} | max cs A {max_cards_saved:>2.0f} | ls {learn_iters:>5} *')
    else:
        episodes_without_best += 1
        
    if episodes_without_best % 100 == 0 and episodes_without_best > 0:
        recent_std = np.std(score_history[-100:])
        print(f'episode {i:>5} | score {score:>6.1f} | avg s {avg_score:>6.1f} | max s R {recent_max:>5.1f} | max s A {max_score:>5.1f} | avg cs {avg_cards_saved:>4.1f} | max cs R {recent_max_cs:>2.0f} | max cs A {max_cards_saved:>2.0f} | ls {learn_iters:>5}')

... loading models ...
New run, godspeed lad
... saving models ...
episode    87 | score    3.7 | avg s    0.5 | max s R   5.7 | max s A   5.7 | avg cs  6.5 | max cs R 41 | max cs A 41 | ls     1 *
... saving models ...
episode    91 | score    3.9 | avg s    0.6 | max s R   5.7 | max s A   5.7 | avg cs  6.7 | max cs R 41 | max cs A 41 | ls     1 *
... saving models ...
episode    96 | score    5.4 | avg s    0.6 | max s R   5.7 | max s A   5.7 | avg cs  6.9 | max cs R 41 | max cs A 41 | ls     1 *
... saving models ...
episode    99 | score    3.3 | avg s    0.6 | max s R   5.7 | max s A   5.7 | avg cs  7.0 | max cs R 41 | max cs A 41 | ls     1 *
... saving models ...
episode   102 | score    1.1 | avg s    0.6 | max s R   5.7 | max s A   5.7 | avg cs  7.1 | max cs R 41 | max cs A 41 | ls     1 *
... saving models ...
episode   103 | score    1.0 | avg s    0.6 | max s R   5.7 | max s A   5.7 | avg cs  7.3 | max cs R 41 | max cs A 41 | ls     1 *
... saving models ...
episode   106 |

In [ ]:
import gymnasium as gym
import numpy as np

env = gym.make('CartPole-v1', render_mode='human')
N = 20
batch_size = 5
n_epochs = 4
alpha = 0.0003
agent = Agent(n_actions=env.action_space.n, batch_size=batch_size, 
                alpha=alpha, n_epochs=n_epochs, 
                input_dims=env.observation_space.shape)
n_games = 300

figure_file = 'plots/cartpole.png'

best_score = -1000
score_history = []
max_score = 0
episodes_without_best = 0


learn_iters = 0
avg_score = 0
n_steps = 0

for i in range(n_games):
    observation = env.reset()[0]
    done = False
    truncated = False
    score = 0
    while not done and not truncated:
        action, prob, val = agent.choose_action(observation)
        observation_, reward, done, truncated, info = env.step(action)
        n_steps += 1
        score += reward
        agent.remember(observation, action, prob, val, reward, done)
        if n_steps % N == 0:
            agent.learn()
            learn_iters += 1
        observation = observation_
    score_history.append(score)
    avg_score = np.mean(score_history[-100:])
    
    max_score = max(max_score, score)
    
    if avg_score > best_score:
        best_score = avg_score
        recent_std = np.std(score_history[-100:])
        agent.save_models()
        episodes_without_best = 0
        print(f'episode {i:>5} | score {score:>6.1f} | avg {avg_score:>6.1f} | std {recent_std:>6.2f} | max score {max_score:>5.1f} | learning steps {learn_iters:>5} | done {done} *')
    else:
        episodes_without_best += 1
        
    if episodes_without_best % 100 == 0 and episodes_without_best > 0:
        recent_std = np.std(score_history[-100:])
        print(f'episode {i:>5} | score {score:>6.1f} | avg {avg_score:>6.1f} | std {recent_std:>6.2f} | max score {max_score:>5.1f} | learning steps {learn_iters:>5} | done {done}')


In [ ]:
env.close()

In [5]:
agent.save_models()

... saving models ...


In [ ]:
demo_env.close()

In [ ]:
%pip install -e ../.